# Calculate VMT per household type

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from datetime import datetime
import replicaEVSE.sql_wrapper_functions as sql
import replicaEVSE.load_curve as sim
import replicaEVSE.vmt as vmt
from dask.distributed import Client, LocalCluster
%reload_ext autoreload
%autoreload 1

datadir = '../../data/'

pd.set_option('display.max_columns', None)

# cluster = LocalCluster(n_workers=63)  # Launches a scheduler and workers locally
# client = Client(cluster)  # Connect to distributed cluster and override default

### add home counties which will work for private_auto but not commercial which aren't in the population data table

In [2]:
df = pd.read_parquet(datadir+'/wa_pop_and_trips_sorted.parquet') # len = 51727268
counties = pd.read_parquet(datadir+'/population_counties_dataset.parquet', engine='pyarrow')

In [3]:
cdf = pd.merge(df, counties, on='person_id', how='left')
cdf

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,home_cty,home_st
0,18336634547462657450,10000002909730821040,PRIVATE_AUTO,SOCIAL,HOME,0 days 08:20:42,0 days 08:39:00,2.897196,nan,530250111002,47.098457,-119.319023,530250111003,47.117839,-119.303234,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.085370,-119.319090,47.118737,-119.298334,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",Washington
1,1694659094182985471,10000002909730821040,PRIVATE_AUTO,HOME,SOCIAL,0 days 09:12:45,0 days 09:19:56,2.867602,nan,530250111003,47.117839,-119.303234,530250111002,47.098457,-119.319023,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.118737,-119.298334,47.085370,-119.319090,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,HOME,"Grant County, WA",Washington
2,7721623352602932778,10000002909730821040,ON_DEMAND_AUTO,SHOP,HOME,0 days 09:30:08,0 days 09:49:04,8.929170,nan,530250111002,47.098457,-119.319023,530250109022,47.152531,-119.293494,residential,single_family,residential,single_family,commercial,retail,commercial,retail,47.085370,-119.319090,47.144694,-119.278788,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",Washington
3,3697919619727259534,10000002909730821040,CARPOOL,SHOP,SHOP,0 days 09:54:21,0 days 09:57:14,0.996870,nan,530250109022,47.152531,-119.293494,530250111001,47.123046,-119.285771,commercial,retail,commercial,retail,commercial,retail,commercial,retail,47.144694,-119.278788,47.131541,-119.276257,thursday,17571687911180552742,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64.0,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA",Washington
4,9977271661694103189,10000002909730821040,WALKING,SHOP,SHOP,0 days 10:23:28,0 da

can see that not all entries have a home blockgroup

In [4]:
cdf.BLOCKGROUP.value_counts(dropna=False)

BLOCKGROUP
None            2156578
530330322103      53832
530630107001      49350
530730001003      48915
530530731251      45678
                 ...   
410390024011          1
410579604006          1
300310015002          1
300979670002          1
410710306012          1
Name: count, Length: 8132, dtype: int64

In [5]:
# change data types for consistency
cdf['home_cty'] = cdf['home_cty'].astype(str)
cdf['home_st'] = cdf['home_st'].astype(str)

## LDV VMT

In [13]:
weekday = 'thursday'
mode = 'PRIVATE_AUTO'
ldv_vmt_thur = vmt.calc_vmt_by_county(cdf, mode=mode, weekday=weekday) # .compute()
ldv_vmt_thur.to_csv(datadir+f'vmt_ldv_{weekday}.csv', index=False, mode='w')
ldv_vmt_thur

,home_cty,distance_miles
0,"Ada County, ID",3.440208e+04
1,"Adams County, ID",7.667243e+02
2,"Adams County, WA",5.665706e+05
3,"Asotin County, WA",2.993855e+05
4,"Baker County, OR",4.936120e+03
...,...,...
131,"Whitman County, WA",7.216444e+05
132,"Yakima County, WA",4.571530e+06
133,"Yamhill County, OR",2.987557e+04
134,"Yellowstone County, MT",2.733071e+02


In [14]:
weekday = 'saturday'

ldv_vmt_sat = vmt.calc_vmt_by_county(cdf, mode='PRIVATE_AUTO', weekday=weekday)
ldv_vmt_sat.to_csv(datadir+f'vmt_ldv_{weekday}.csv', index=False, mode='w')
ldv_vmt_sat.head()

,home_cty,distance_miles
0,"Ada County, ID",28044.363543
1,"Adams County, ID",32.248312
2,"Adams County, WA",563222.231352
3,"Asotin County, WA",211291.833937
4,"Baker County, OR",1575.514290


## HDV VMT

These don't have a home county associated with them so will have to use the desitnation blockgroup

In [6]:
# read in blockgroup info
bg_df = pd.read_csv(datadir+'blockgroup_counties.csv')
bg_df['destination_bgrp'] = bg_df.destination_bgrp.astype(str)
bg_df['destination_county'] = bg_df.County.astype(str)

In [7]:
bg_df

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,County,BLOCKGROUP,destination_bgrp,destination_county
0,53,29,971400,2,530299714002,Block Group 2,G5030,S,2814636,1797525,48.206277,-122.529653,"POLYGON ((-122.54657 48.208011, -122.542498 48...",Island County,530299714002,530299714002,Island County
1,53,29,971400,3,530299714003,Block Group 3,G5030,S,22645103,62368,48.212509,-122.499121,"POLYGON ((-122.527323 48.182455, -122.527269 4...",Island County,530299714003,530299714003,Island County
2,53,29,971500,1,530299715001,Block Group 1,G5030,S,14190551,3407670,48.252358,-122.456154,"POLYGON ((-122.485032 48.255072999999996, -122...",Island County,530299715001,530299715001,Island County
3,53,29,971500,2,530299715002,Block Group 2,G5030,S,13421381,9435655,48.214561,-122.452643,"POLYGON ((-122.481784 48.167791, -122.481692 4...",Island County,530299715002,530299715002,Island County
4,53,29,971600,1,530299716001,Block Group 1,G5030,S,13176547,2594118,48.171603,-122.513752,POLYGON ((-122.53916699999999 48.1904959999999...,Island County,530299716001,530299716001,Island County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,53,15,1100,2,530150011002,Block Group 2,G5030,S,311974,41091,46.148888,-122.909208,"POLYGON ((-122.91382899999999 46.14731, -122.9...",Cowlitz County,530150011002,530150011002,Cowlitz County
5339,53,15,1100,3,530150011003,Block Group 3,G5030,S,651187,0,46.139811,-122.902721,"POLYGON ((-122.911793 46.137749, -122.911517 4...",Cowlitz County,530150011003,530150011003,Cowlitz County
5340,53,15,1100,4,530150011004,Block Group 4,G5030,S,220828,0,46.138142,-122.912800,"POLYGON ((-122.91549 46.133441999999995, -122....",Cowlitz County,530150011004,530150011004,Cowlitz County
5341,53,15,1100,5,530150011005,Block Group 5,G5030,S,301290,0,46.136023,-122.908029,"POLYGON ((-122.91351 46.13303, -122.913243 46....",Cowlitz County,530150011005,530150011005,Cowlitz County


### there are two types of commercial traffic

In [8]:
cdf['vehicle_type'].value_counts()

vehicle_type
nan                  50274818
MEDIUM_COMMERCIAL     1335515
HEAVY_COMMERCIAL       116935
Name: count, dtype: int64

## Heavy duty VMT into destination county

In [9]:
weekday = 'thursday'
mode = 'COMMERCIAL'
vehicle_type = 'HEAVY_COMMERCIAL'
hdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
hdvs_cnty = pd.merge(hdvs, bg_df, on='destination_bgrp', how='left')
hdv_vmt = vmt.calc_vmt_by_county(hdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
# hdv_vmt.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

In [11]:
weekday = 'saturday'
mode = 'COMMERCIAL'
vehicle_type = 'HEAVY_COMMERCIAL'
hdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
hdvs_cnty = pd.merge(hdvs, bg_df, on='destination_bgrp', how='left')
hdv_vmt = vmt.calc_vmt_by_county(hdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
hdv_vmt.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

In [13]:
cdf['vehicle_type'].value_counts()

vehicle_type
nan                  50274818
MEDIUM_COMMERCIAL     1335515
HEAVY_COMMERCIAL       116935
Name: count, dtype: int64

## MDV VMT in destinatation county

In [14]:
weekday = 'saturday'
mode = 'COMMERCIAL'
vehicle_type = 'MEDIUM_COMMERCIAL'
mdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
mdvs_cnty = pd.merge(mdvs, bg_df, on='destination_bgrp', how='left')
mdv_vmt_sat = vmt.calc_vmt_by_county(mdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
mdv_vmt_sat.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

## HDV VMT

These don't have a home county associated with them so will have to use the desitnation blockgroup

In [27]:
mdv_vmt_sat

vehicle_type
nan                  50274818
MEDIUM_COMMERCIAL     1335515
HEAVY_COMMERCIAL       116935
Name: count, dtype: int64

In [16]:
weekday = 'thursday'
mode = 'COMMERCIAL'
vehicle_type = 'MEDIUM_COMMERCIAL'
mdvs = cdf.loc[cdf['vehicle_type'] == vehicle_type]
# merge with the blockgroup county info
mdvs_cnty = pd.merge(mdvs, bg_df, on='destination_bgrp', how='left')
mdv_vmt_thu = vmt.calc_vmt_by_county(mdvs_cnty, mode=mode, weekday=weekday, county_key='destination_county')
mdv_vmt_thu.to_csv(datadir+f'vmt_hdv_{vehicle_type}_{weekday}.csv', index=False)

In [17]:
mdv_vmt_thu

,destination_county,distance_miles
0,Adams County,1.730667e+05
1,Asotin County,1.264568e+04
2,Benton County,3.309011e+05
3,Chelan County,1.352807e+05
4,Clallam County,1.015500e+05
5,Clark County,5.925547e+05
6,Columbia County,1.047632e+04
7,Cowlitz County,3.106777e+05
8,Douglas County,8.119115e+04
9,Ferry County,1.314956e+04


## BUSES?

I don't think there is anything we can do here...

# Single-vs multi unit housing

In [39]:
pop = cdf.dropna(subset='home_cty')

In [40]:
pop = pop[pop['home_cty'].str.contains(', WA')].reset_index(drop=True)

In [37]:
pop.home_cty.value_counts(dropna=False)

home_cty
King County, WA         13887885
Pierce County, WA        6098929
Snohomish County, WA     5515824
Spokane County, WA       3447020
Clark County, WA         2929258
                          ...   
Converse County, WY            0
Polk County, OR                0
Coos County, OR                0
Curry County, OR               0
Granite County, MT             0
Name: count, Length: 200, dtype: int64

In [33]:
pop.value_counts('home_cty')


home_cty
King County, WA         13887885
Pierce County, WA        6098929
Snohomish County, WA     5515824
Spokane County, WA       3447020
Clark County, WA         2929258
                          ...   
Converse County, WY            0
Polk County, OR                0
Coos County, OR                0
Curry County, OR               0
Granite County, MT             0
Name: count, Length: 200, dtype: int64

home_cty
King County, WA         13887885
Pierce County, WA        6098929
Snohomish County, WA     5515824
Spokane County, WA       3447020
Clark County, WA         2929258
                          ...   
Converse County, WY            0
Polk County, OR                0
Coos County, OR                0
Curry County, OR               0
Granite County, MT             0
Name: count, Length: 200, dtype: int64

In [50]:


# pop_thu.groupby('home_cty')['building_type'].value_counts()

In [63]:
weekday='saturday'

def home_type_by_county(cdf, weekday):

    # make the data a little smaller
    pop = cdf[cdf['weekday'] == weekday]

    # get unique number of households
    pop = pop.drop_duplicates(subset=['household_id'])
    
    # only include poeple with a home county, ie, no Commercail drivers
    pop = pop.dropna(subset='home_cty')


    pop_thu_single = pop[pop['building_type'] == 'single_family']
    # dont include mobile homes or RVs
    pop_thu_multi = pop[(pop['building_type'] != 'single_family') & (pop['building_type'] != 'mobile')]

    num_single_per_county = pop_thu_single.groupby('home_cty')['household_id'].count()
    num_multi_per_county  = pop_thu_multi.groupby('home_cty')['household_id'].count()

    # create the output table
    out = pd.DataFrame({'single': num_single_per_county, 'multi': num_multi_per_county, 'single/multi': num_single_per_county/num_multi_per_county})
    out = out[out.index.str.contains(', WA')].reset_index()
    return out

In [64]:

out = home_type_by_county(cdf, weekday)
out.to_csv(datadir+f'households_counts_by_county_.csv', index=False)

In [47]:
# Example DataFrame
df = pd.DataFrame({
    'Person_ID': [1, 2, 3, 4, 5],
    'Home_County': ['Spokane, WA', 'King, WA', 'Pierce, OR', 'Kitsap, WY', np.nan],
    'Age': [35, 42, 28, 39, 55]
})

df = df.dropna(subset=['Home_County'])
# Filter the DataFrame where 'Home_County' contains ', WA'
filtered_df = df[df['Home_County'].str.contains(', WA')]

# Print the filtered DataFrame
filtered_df.value_counts('Home_County')

Home_County
King, WA       1
Spokane, WA    1
Name: count, dtype: int64